In [19]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI
from llama_index.core import SQLDatabase
from llama_index.core import Settings

from sqlalchemy import create_engine
import os
from IPython.display import Markdown, display

Configurando parametros com relação ao modelo LLM utilizado

In [12]:
os.environ["OPENAI_API_KEY"] = "sk-SUA+CHAVE"
model = "gpt-3.5-turbo"
chunk_size = 2048
Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
Settings.chunk_size = chunk_size

Conectamos ao nosso banco de dados, para fins de estudos usaremos o SQLite mas esta conexão pode ser substituida por qualquer conexão usando o SQLAlchemy

In [30]:
!wget https://github.com/JoseLuckmann/tutorial-llama-index/raw/main/rh.db

--2024-04-08 11:14:07--  https://github.com/JoseLuckmann/tutorial-llama-index/raw/main/rh.db
Resolving github.com (github.com)... 20.201.28.151
Connecting to github.com (github.com)|20.201.28.151|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JoseLuckmann/tutorial-llama-index/main/rh.db [following]
--2024-04-08 11:14:08--  https://raw.githubusercontent.com/JoseLuckmann/tutorial-llama-index/main/rh.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 176128 (172K) [application/octet-stream]
Saving to: ‘rh.db’

rh.db               100%[===================>] 172.00K  --.-KB/s    in 0.07s   

2024-04-08 11:14:08 (2.27 MB/s) - ‘rh.db’ saved [176128/176128]



In [31]:
connection_string = "sqlite:///rh.db"
engine = create_engine(connection_string)

Agora conectamos o nosso banco de dados ao Index aqui surgem alguns objetos novos que é importante entendermos

- SQLDatabase: Uma abstração simples para permitir a conexão entre o SQLAlchemy e o LLamaIndex
- NLSQLTableQueryEngine: Motor de consulta de tabelas SQL em linguagem natural
- QueryEngineTool: Uma ferramenta que utiliza um motor de pesquisa (no nosso caso a NLSQLTable)
- SubQuestionQueryEngine? Divide uma consulta complexa em muitas sub-perguntas e no seu motor de consulta de destino para execução. Após a execução de todas as sub-perguntas, todas as respostas são reunidas e enviadas para o sintetizador de respostas para produzir a resposta final.

In [32]:
sql_database = SQLDatabase(engine)
sql_query_engine = NLSQLTableQueryEngine(sql_database=sql_database, verbose=True)
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description="""Utilizado para traduzir a linguagem natural em linguagem SQL para executar as querys
Base de dados de recursos humanos com informações de funcionarios, salarios, vagas e etc""",
)

In [16]:
query_engine = SubQuestionQueryEngine.from_defaults(
        query_engine_tools=[sql_tool], #No nosso caso temos apenas uma conexão, mas aqui é possivel informar varias conexões
    )

query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="sub_question_query_engine",
        description="Util para responder perguntas sobre as bases de dados que você tem acesso.",
    ),
)

tools = [query_engine_tool] + [sql_tool]

Iniciamos um agente da OpenAI com acesso as nossas ferramentas

In [21]:
agent = OpenAIAgent.from_tools(tools, verbose=False)

Vamos testar algumas perguntas agora

In [27]:
response = agent.chat("Quantos funcionários eu tenho?")
display(Markdown(str(response)))

Você tem um total de 1000 funcionários.

In [26]:
response = agent.chat("Quem é o funcionário com maior salário?")
display(Markdown(str(response)))

O funcionário com o maior salário é Emily Lewis, com um salário de $9986.00.

In [25]:
response = agent.chat("Crie uma tabela com a soma da minha folha salarial mes a mes dos ultimos 6 meses")
display(Markdown(str(response)))

Aqui está a tabela com a soma da folha salarial mês a mês dos últimos 6 meses:

- Janeiro: $498,353.00
- Fevereiro: $570,587.00
- Março: $507,059.00
- Abril: $422,937.00
- Maio: $464,552.00
- Junho: $560,706.00